In [ ]:
pip install pytesseract

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install pydicom

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Instruction to install and set up pytesseract (OCR tool for Python) is following:
# https://pypi.org/project/pytesseract/
# It might be necessary to download pytesseract.pytesseract.tesseract_cmd and put it in your data directory

In [ ]:
import os
import re
import shutil
import pydicom
import numpy as np
import matplotlib.pyplot as plt
import pytesseract
import cv2

In [ ]:
# This will need different directory
pytesseract.pytesseract.tesseract_cmd = r'D:/UPENN/Research/Data/Tesseract-OCR/tesseract.exe' 

In [ ]:
paths = [] #List of directories for inidividual patients

for r,d,f in os.walk("D:/UPENN/Research/Data_testing"):
    for directory in d:
        if directory[0] == "1":
            paths.append(os.path.join(r,directory,""))
            
for idx in list(range(len(paths))):
    paths[idx] = paths[idx].replace("\\", "/")

In [ ]:
for path in paths:
    count = 0
    
    # Changing directory
    os.chdir(path)
    print(path)                            # Print a directory for individual patients
    
    #Creating directories for organs
#     os.mkdir("Ureter")
#     os.mkdir("Kidney")
#     os.mkdir("Bladder")
#     os.mkdir("Unknown")
    
#     ureter_path = os.path.join(path,'Ureter')
#     kidney_path = os.path.join(path,'Kidney')
#     bladder_path = os.path.join(path,'Bladder')
#     unknown_path = os.path.join(path,'Unknown')
    
    # Text Extraction
    for r,d,f in os.walk(path):
        for file in f:
            if ".dcm" in file:
                count += 1
                data = pydicom.dcmread(os.path.join(r,file))
                arr = data.pixel_array
                shape = arr.shape               # Depending on the file, shape varies from 2 to 4. Most DICOM files have shape of 2. 
                if len(shape) == 4:            # In other cases, assuming that the text informations are consistant, I use first slide.
                    i, x, y, z = arr.shape
                    arr = arr[0,:,:,0]
                elif len(shape) == 3:
                    x, y, z = arr.shape
                    arr = arr[:,:,0]
                else:
                    x, y = arr.shape
                img = cv2.cvtColor(arr, cv2.COLOR_GRAY2BGR)
                (thresh, bw) = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV)   
                bw = bw[int(x*0.86)::,:]         # Can change the area of interest in the image with different value (0.86 in this case)
                text = pytesseract.image_to_string(bw,lang='eng')
                text = text.split()
                for i in list(range(len(text))):
                    text[i] = text[i].lower()
                    text[i] = re.sub(r"[^A-Za-z]+", '', text[i])
                text = [i for i in text if i] 
                
                # Move files based on the text results
                if 'kidney' in text:
                    shutil.move(file, kidney_path)
                elif 'ureter' in text:
                    shutil.move(file,ureter_path)
                elif 'bladder' in text:
                    shutil.move(file,bladder_path)
                else:
                    shutil.move(file,unknown_path)
        break
    
    print("Number of files: ", count)         # Print number of files in each patients

In [ ]:
# This is a testing code with figure of area of interest and extracted text results

for r,d,f in os.walk(paths[6]):
    for file in f:
        if ".dcm" in file:
            data = pydicom.dcmread(os.path.join(r,file))
            arr = data.pixel_array
            shape = arr.shape
            if len(shape) == 4:
                i, x, y, z = arr.shape
                arr = arr[0,:,:,0]
            elif len(shape) == 3:
                x, y, z = arr.shape
                arr = arr[:,:,0]
            else:
                x, y = arr.shape
            img = cv2.cvtColor(arr, cv2.COLOR_GRAY2BGR)
            (thresh, bw) = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV)
            bw = bw[int(x*0.85)::,:]
            text = pytesseract.image_to_string(bw,lang='eng')
            text = text.split()
            for i in list(range(len(text))):
                text[i] = text[i].lower()
                text[i] = re.sub(r"[^A-Za-z]+", '', text[i])
            text = [i for i in text if i] 
            print(text)
            plt.imshow(bw, cmap="gray")
            plt.show()
            print("")